In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mplt
import copy
import csv
from ESIOS import *
from matplotlib import pyplot as plt
from datetime import datetime
from scipy.stats import pearsonr
from functools import reduce

def variables(fecha_inicio, fecha_fin):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    
    indicadores = [1, 12, 15, 20, 22, 24, 33, 36, 50, 57, 58, 59, 68, 71, 77, 79, 86, 89, 91, 92, 93, 96, 98, 105, 106, 107, 109, 113, 114, 117, 122, 123, 124, 127, 128, 129, 138, 141, 142, 147, 148, 152, 156, 157, 158, 160, 162, 173, 175, 176, 183, 184, 191, 192, 194, 197, 198, 201, 359, 360, 368, 371, 372, 373, 374, 380, 386, 387, 421, 454, 463, 480, 489, 492, 495, 496, 497, 498, 500, 502, 505, 510, 512, 514, 515, 516, 519, 522, 524, 528, 529, 530, 531, 532, 533, 534, 537, 538, 541, 572, 578, 579, 632, 633, 634, 674, 675, 680, 684, 685, 704, 709, 712, 726, 727, 734, 735, 738, 742, 750, 751, 754, 755, 757, 762, 764, 766, 770, 775, 776, 777, 778, 780, 781, 783, 784, 785, 786, 788, 789, 793, 794, 795, 797, 798, 802, 803, 804, 805, 806, 807, 813, 815, 816, 830, 843, 899, 1014, 1015, 1017, 1019, 1020, 1024, 1025, 1026, 1029, 1031, 1032, 1033, 1034, 1035, 1045, 1048, 1074, 1075, 1079, 1084, 1085, 1086, 1088, 1106, 1113, 1114, 1115, 1116, 1136, 1173, 1177, 1180, 1181, 1189, 1192, 1196, 1197, 1198, 1202, 1204, 1212, 1213, 1216, 1217, 1275, 1276, 1277, 1286, 1334, 1338, 1341, 1342, 1343, 1344, 1349, 1350, 1352, 1354, 1358, 1369, 1373, 1376, 1389, 1397, 1411, 1420, 1421, 1422, 1423, 1425, 1434, 1437, 1438, 1635, 1636, 1637, 1638, 1641, 1642, 1645, 1646, 1647, 1651, 1652, 1656, 1657, 1661, 1663, 1666, 1686, 1687, 1727, 1775, 1776, 1777, 1788, 10008, 10010, 10012, 10013, 10018, 10021, 10023, 10024, 10026, 10027, 10031, 10034, 10047, 10051, 10055, 10056, 10063, 10064, 10067, 10068, 10073, 10086, 10088, 10089, 10095, 10096, 10098, 10105, 10106, 10107, 10108, 10114, 10116, 10133, 10134, 10159, 10169, 10171, 10176, 10177, 10186, 10187, 10188, 10189, 10190, 10207, 10208, 10209, 10211, 10212, 10222, 10223, 10228, 10229, 10230, 10235, 10236, 10241, 10242, 10247, 10249, 10250, 10257, 10258, 10267, 10268, 10270, 10271, 10274, 10275, 10276, 10281, 10286, 10287, 10321, 10324, 10359, 10364, 10380, 10381]
    lista = []
    start = datetime.strptime(fecha_inicio, '%Y-%m-%d')
    end = datetime.strptime(fecha_fin, '%Y-%m-%d')
    dias = end - start
    objetivo = dias.days*24+2  #Puede ser +1 +2 dependiendo del cambio horario
    print('objetivo=' +str(objetivo))
    for i in indicadores:
        datos = esios.get_data(i, start, end)
        if(datos is not None):
            print(str(i) + '=' + str(len(datos)))
            if(len(datos)==objetivo):
                lista.append(i)
    print('Tienes ' + str(len(lista)) + ' variables validas')
    return lista

def nombres_indicadores(indicadores):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    lista_nombres = esios.get_names(indicadores)
    lista_final = dict(zip(lista_nombres,indicadores))
    df = pd.DataFrame.from_dict(lista_final, orient='index')
    df = df.transpose()
    df.to_csv("nombres_indicadores.csv", index=False)

def extraer_datos(fecha_inicio, fecha_fin, nombre, indicadores = []):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    
    start = datetime.strptime(fecha_inicio, '%Y-%m-%d')
    end = datetime.strptime(fecha_fin, '%Y-%m-%d')
   
    if(not indicadores):

        indicadores = variables(fecha_inicio, fecha_fin)
    
    df_list, names = esios.get_multiple_series(indicadores, start, end)
    
    final = esios.merge_series(df_list, names, pandas_sampling_interval='1H')
    final.to_csv(nombre+'.csv')

def extraer_datos_por_partes(fecha_inicio, fecha_fin, nombre, indicadores = []):
    corte = 5    #Número de variables en cada subgrupo
    lista_indicadores = []
    lista_df = []
    i=0
    e=0
 
    if(not indicadores):
        indicadores = variables(fecha_inicio, fecha_fin)

    lista_indicadores=[indicadores[i:i + corte] for i in range(0, len(indicadores), corte)]
    
    if(len(lista_indicadores[-1])<corte):
        lista_indicadores[-2]=lista_indicadores[-2]+lista_indicadores[-1]
        lista_indicadores.pop()
    print(lista_indicadores)
    
    for i in lista_indicadores:
        nombre_parte = 'Extraer parte '+ str(e)
        extraer_datos(fecha_inicio, fecha_fin, nombre_parte , i)
        e=e+1
        
    for i in range(0,len(lista_indicadores)):
        parte_leer = 'Extraer parte ' + str(i) + '.csv'
        data = pd.read_csv(parte_leer)
        if(i!=0):
            del(data['datetime_utc'])
        lista_df.append(data)  

    df = pd.concat(lista_df,axis=1, join='inner')

    pd.DataFrame.to_csv(df, nombre+'.csv', sep=',', na_rep='.', index=False)
    print('Fin de extraer datos por partes')  
    
def seleccion_variables_pearson(nombre):
    token = '079272e1d20f8f1e79d1b886d49050d30534786e9ecc483b7835ff427a0d798f'
    esios = ESIOS(token)
    
    df = pd.read_csv(nombre+".csv")
    nombres = pd.read_csv("nombres_indicadores.csv")
    columnas = df.columns
    objetivo = df['Precio medio horario final suma de componentes']
    lista04 = []
    lista06 = []
    lista075 = []
    for nom in columnas:
        if(not nom=='datetime_utc'):
            columna = df[nom]
            corr, _ = pearsonr(objetivo, columna)
            print('%.3f' % corr+' : '+nom)
            if(abs(corr) >= 0.75):
                ind = nombres.get(nom)
                lista075.append(int(ind[0]))
                lista06.append(int(ind[0]))
                lista04.append(int(ind[0]))
            elif(abs(corr) >= 0.6):
                ind = nombres.get(nom)
                lista06.append(int(ind[0]))
                lista04.append(int(ind[0]))
            elif(abs(corr) >= 0.4):
                ind = nombres.get(nom)
                lista04.append(int(ind[0]))
    return lista075, lista06, lista04

def MAE (test_y, predicciones):
    errores=[]
    for i in range(0, len(test_y)):
        errores.append(abs(test_y[i]-predicciones[i]))
    return np.mean(errores)

def MAPE (test_y, predicciones):
    errores=[]
    for i in range(0, len(test_y)):
        errores.append(abs(test_y[i]-predicciones[i])/test_y[i]*100)
    return np.mean(errores)

def WAPE (test_y, predicciones): 
    actual=sum(test_y)
    pred=sum(predicciones)   
    return float(abs(actual-pred)/actual*100)

def resultado(nombre, algoritmo, num):
    MAE=[]
    MAPE=[]
    WAPE=[]
    for i in range(0,num):
        print("Iteración "+str(i+1))
        resultadoMAE,resultadoMAPE,resultadoWAPE=algoritmo(nombre)
        MAE.append(resultadoMAE)
        MAPE.append(resultadoMAPE)
        WAPE.append(resultadoWAPE)
    resultado=[np.mean(MAE),np.mean(MAPE),np.mean(WAPE)]
    print("================================")
    print("Media final: \nError MAE: "+str(resultado[0])+"\nError MAPE: "+str(resultado[1])+"\nError WAPE: "
          +str(resultado[2]))
    return resultado 

In [5]:
nombres_indicadores([2, 4, 14, 16, 17, 18, 19, 21, 23, 26, 35, 37, 39, 43, 44, 47, 49, 51, 52, 53, 54, 55, 56, 61, 70, 72, 74, 78, 82, 87, 88, 94, 103, 112, 120, 121, 125, 126, 131, 140, 144, 149, 155, 159, 161, 163, 164, 166, 177, 179, 182, 187, 190, 193, 195, 196, 199, 208, 210, 351, 352, 353, 354, 358, 361, 365, 366, 367, 375, 379, 381, 382, 388, 389, 432, 453, 455, 456, 457, 459, 460, 462, 467, 470, 475, 476, 478, 485, 488, 490, 491, 493, 494, 499, 501, 503, 504, 506, 507, 511, 513, 517, 518, 520, 521, 523, 525, 526, 527, 542, 543, 549, 551, 553, 570, 571, 574, 605, 606, 607, 613, 614, 615, 630, 631, 666, 667, 681, 683, 686, 687, 701, 705, 708, 710, 713, 719, 737, 739, 740, 752, 759, 763, 765, 767, 768, 769, 771, 774, 779, 782, 787, 790, 791, 792, 796, 799, 800, 801, 808, 809, 810, 811, 812, 814, 817, 824, 825, 829, 832, 833, 837, 839, 841, 842, 844, 845, 846, 856, 1013, 1016, 1018, 1021, 1022, 1023, 1027, 1028, 1030, 1036, 1046, 1047, 1073, 1076, 1077, 1078, 1080, 1081, 1082, 1083, 1087, 1105, 1120, 1152, 1156, 1161, 1162, 1163, 1165, 1166, 1167, 1168, 1171, 1172, 1175, 1176, 1178, 1183, 1187, 1188, 1193, 1195, 1201, 1203, 1205, 1206, 1207, 1210, 1211, 1218, 1238, 1243, 1244, 1245, 1247, 1253, 1284, 1285, 1293, 1294, 1296, 1297, 1335, 1336, 1339, 1340, 1347, 1348, 1351, 1353, 1357, 1366, 1367, 1368, 1388, 1390, 1398, 1399, 1400, 1401, 1403, 1406, 1407, 1408, 1415, 1416, 1417, 1418, 1427, 1432, 1435, 1436, 1442, 1632, 1633, 1634, 1639, 1640, 1643, 1644, 1648, 1649, 1650, 1653, 1654, 1655, 1658, 1659, 1660, 1662, 1664, 1665, 1667, 1682, 1683, 1684, 1685, 1690, 1691, 1739, 1744, 1745, 1747, 1750, 1751, 1752, 1754, 1778, 1779, 1780, 1787, 1789, 1794, 1820, 1821, 10001, 10003, 10004, 10011, 10014, 10015, 10017, 10019, 10020, 10022, 10028, 10029, 10030, 10032, 10033, 10037, 10040, 10041, 10042, 10043, 10046, 10050, 10054, 10057, 10059, 10065, 10066, 10074, 10087, 10090, 10097, 10099, 10104, 10113, 10115, 10117, 10131, 10132, 10136, 10141, 10142, 10143, 10144, 10145, 10158, 10160, 10161, 10168, 10170, 10178, 10179, 10195, 10205, 10206, 10211, 10214, 10220, 10221, 10231, 10232, 10237, 10238, 10243, 10244, 10245, 10246, 10248, 10251, 10252, 10254, 10255, 10256, 10259, 10260, 10261, 10262, 10269, 10273, 10278, 10279, 10282, 10283, 10284, 10323, 10325, 10326, 10327, 10351, 10352, 10354, 10356, 10358, 10377, 10378, 10379, 10382, 10383])

In [1]:
##Primer Periodo (baseline)
extraer_datos('2018-03-01', '2018-05-31', 'Primer periodo')
lista075, lista06, lista04 = seleccion_variables_pearson('Primer periodo')
extraer_datos('2018-03-01', '2018-05-31', 'Primer periodo 075', lista075)
extraer_datos('2018-03-01', '2018-05-31', 'Primer periodo 06', lista06)
extraer_datos('2018-03-01', '2018-05-31', 'Primer periodo 04', lista04)

In [ ]:
##Segundo periodo (primera cuarentena)
extraer_datos('2020-03-01', '2020-05-31', 'Segundo periodo')
lista075, lista06, lista04 = seleccion_variables_pearson('Segundo periodo')
extraer_datos('2020-03-01', '2020-05-31', 'Segundo periodo 075', lista075)
extraer_datos('2020-03-01', '2020-05-31', 'Segundo periodo 06', lista06)
extraer_datos('2020-03-01', '2020-05-31', 'Segundo periodo 04', lista04)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from tcn import TCN, tcn_full_summary
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot


def random_forest(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
        
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    
    modelo = RandomForestRegressor(n_estimators = 30, criterion ='mse', max_depth = None, max_features = 'auto',
                                   oob_score = False,n_jobs = -1)

    modelo.fit(train_X, train_y)

    predicciones = modelo.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    return errorMAE,errorMAPE,errorWAPE

def multilayer_perceptron(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
    
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]
    
    modelo = MLPRegressor(hidden_layer_sizes=(32,32,32,32,32), activation="relu", random_state=1, max_iter=2000)
    modelo.fit(train_X, train_y)

    predicciones = modelo.predict(test_X)
    
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    return errorMAE,errorMAPE,errorWAPE

def tcn(nombre):
    df = pd.read_csv(nombre+".csv",index_col="datetime_utc")
    
    values = df.values
    names = df.columns
    indice = names.get_loc("Precio medio horario final suma de componentes")

    values = values.astype('float32')

    division = len(values)*66//100
    train = values[:division, :]
    test = values[division:, :]

    train_y = train[:, indice]
    train = np.delete(train, indice, 1) 
    train_X = train[:, :]
    test_y = test[:, indice]
    test = np.delete(test, indice, 1) 
    test_X = test[:, :]

    time_steps, input_dim = 20, 1
    tcn_layer = TCN(nb_filters=60, input_shape = (time_steps, input_dim))

    modelo = Sequential([tcn_layer, Dense(1)])

    modelo.compile(optimizer='adam', loss='mean_absolute_error')
    history = modelo.fit(train_X, train_y, epochs=150, validation_data=(test_X, test_y))

    predicciones = modelo.predict(test_X)
    errorMAE=MAE(test_y, predicciones)
    print("Error MAE: "+str(errorMAE))
     
    errorMAPE=MAPE(test_y, predicciones)
    print("Error MAPE: "+str(errorMAPE))

    errorWAPE=WAPE(test_y, predicciones)
    print("Error WAPE: "+str(errorWAPE))
    
    print(modelo.summary())

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()
    
    return errorMAE,errorMAPE,errorWAPE

In [4]:
random_forest("Segundo periodo 075")
multilayer_perceptron("Segundo periodo 075")
tcn("Segundo periodo 075")

In [10]:
resultado("Segundo periodo 04",random_forest, 5)

In [ ]:
algoritmos=[random_forest, multilayer_perceptron, tcn]
periodos=["Primer periodo 075","Primer periodo 06", "Primer periodo 04", 
          "Segundo periodo 075","Segundo periodo 06","Segundo periodo 04"]

resultados=[]
for i in periodos:
    resultados.append(i)
    for k in algoritmos:
        resultados.append(str(k)+str(resultado(i,k,30)))

print("=====================Resultados=====================")
for i in resultados:
    print(i)